# Ataque de evasión
---

#### RobertoFigueroa, Michele Benvenuto, Gustavo Méndez
Universidad Del Valle de Guatemala \Security Data Science \Proyecto 3 

---

## Modelo original

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, 
    MaxPooling2D,
    Dense,
    Rescaling,
    Flatten,
    Dropout
)

from keras.models import Sequential

from sklearn.metrics import (
    confusion_matrix, 
    precision_score, 
    recall_score, 
    f1_score, 
    plot_confusion_matrix, 
    accuracy_score
)

from art.estimators.classification import KerasClassifier
from sklearn.model_selection import train_test_split

import os
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
if tf.executing_eagerly():
    tf.compat.v1.disable_eager_execution()

c:\users\hp\appdata\local\programs\python\python38\lib\site-packages\art\estimators\certification\__init__.py:12: UserWarning: PyTorch not found. Not importing DeepZ functionality
  warnings.warn("PyTorch not found. Not importing DeepZ functionality")


In [2]:
def get_train_test(data_dir, batch_size=32, img_height=64, img_width=64):
    
    ig = tf.keras.preprocessing.image.ImageDataGenerator()
    data = ig.flow_from_directory(directory=data_dir, 
                                     target_size=(img_height, img_width), 
                                     batch_size=batch_size,
                                     seed=123)

    X, y = data.next()
    
    return X, y

In [3]:
def load_model(img_height, img_width, num_classes):
    model = Sequential([
    Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes)
    ])
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer="adam",
                                                                  metrics=['accuracy'])
    
    return model

In [4]:
def get_metrics(y_val, predictions, model_name):
    conf_matrix = confusion_matrix(y_val, predictions, labels=list(range(1,26)))
    accu = accuracy_score(y_val, predictions)
    _recall_score = recall_score(y_val, predictions, average='weighted')
    _precision_score = precision_score(y_val, predictions, average='weighted')
    _f1_score = f1_score(y_val, predictions, average='weighted')

    print(f"\n---Metrics for {model_name}---")
    #print("\n Confusion Matrix\n", conf_matrix)
    print("\n Accuracy", accu)
    print("\n Precision", _precision_score)
    print("\n Recall", _recall_score)
    print("\n F1 Score", _f1_score)
    
    return conf_matrix

### Entrenando el modelo original


In [5]:
DIR = './data/malimg_paper_dataset_imgs/' 
batch_size=9339
img_height=64
img_width=64
X, y = get_train_test(DIR, batch_size, img_height, img_width)

Found 9339 images belonging to 25 classes.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=79)

In [7]:
print(X_test.shape)
print(y_test.shape)

(1868, 64, 64, 3)
(1868, 25)


In [8]:
model = load_model(img_height, img_width, y.shape[1])

In [9]:
epochs=10
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    batch_size=32
)

Train on 7471 samples
Epoch 1/10
7471/7471 [==============================] - 20s 3ms/sample - loss: 0.9375 - accuracy: 0.6991
Epoch 2/10
7471/7471 [==============================] - 21s 3ms/sample - loss: 0.2360 - accuracy: 0.9300
Epoch 3/10
7471/7471 [==============================] - 22s 3ms/sample - loss: 0.1159 - accuracy: 0.9649
Epoch 4/10
7471/7471 [==============================] - 22s 3ms/sample - loss: 0.0711 - accuracy: 0.9775
Epoch 5/10
7471/7471 [==============================] - 22s 3ms/sample - loss: 0.0515 - accuracy: 0.9839
Epoch 6/10
7471/7471 [==============================] - 21s 3ms/sample - loss: 0.0438 - accuracy: 0.9850
Epoch 7/10
7471/7471 [==============================] - 21s 3ms/sample - loss: 0.0310 - accuracy: 0.9896
Epoch 8/10
7471/7471 [==============================] - 21s 3ms/sample - loss: 0.0230 - accuracy: 0.9913
Epoch 9/10
7471/7471 [==============================] - 21s 3ms/sample - loss: 0.0186 - accuracy: 0.9929
Epoch 10/10
7471/7471 [==========

### Pruebas modelo original

In [10]:
print(model.evaluate(X_test, y_test))

[0.07416157039307153, 0.97858673]


## Evasión del modelo

In [11]:
from art.attacks.evasion import FastGradientMethod

### Creando las observaciones falsas

In [12]:
og_classifier = KerasClassifier(model=model)

In [13]:
epsilon = 10
adv_crafter = FastGradientMethod(og_classifier, eps=epsilon)
x_test_adv = adv_crafter.generate(x=X_test)

### Utilizando las observaciones falsas en el modelo

In [14]:
preds = np.argmax(og_classifier.predict(x_test_adv), axis =1)
acc = np.sum(preds == np.argmax(y_test,axis =1))/ y_test.shape[0]
print("\nTest accuracy on adversarial sample: %.2f%%" % (acc * 100))


Test accuracy on adversarial sample: 15.36%


Se puede observar que el desempeño del modelo es considerablemete peor al momento de analizar imagenes con ruido, para proteger nuestro modelo ante este tipo de ataque podriamos entrenar nuestro modelo con imagenes con ruido.

## Entrenando un nuevo modelo incluyendo imagenes con ruido

In [15]:
x_arrays = np.array_split(X_train, 3)
y_arrays = np.array_split(y_train,3)

In [16]:
for i in y_arrays:
    y_train = np.concatenate([y_train, i])

In [17]:
epsilons = [5, 10, 15]
for i in range(3):
    adv_crafter = FastGradientMethod(og_classifier, eps=epsilons[i])
    x_test_adv = adv_crafter.generate(x=x_arrays[i])
    X_train = np.concatenate([X_train, x_test_adv])

In [19]:
new_model = load_model(img_height, img_width, y.shape[1])

In [20]:
epochs=10
history = new_model.fit(
    x=X_train,
    y=y_train,
    epochs=epochs,
    batch_size=32
)

Train on 14942 samples
Epoch 1/10
14942/14942 [==============================] - 41s 3ms/sample - loss: 0.6290 - accuracy: 0.8054
Epoch 2/10
14942/14942 [==============================] - 43s 3ms/sample - loss: 0.1338 - accuracy: 0.9556
Epoch 3/10
14942/14942 [==============================] - 42s 3ms/sample - loss: 0.0698 - accuracy: 0.9763
Epoch 4/10
14942/14942 [==============================] - 41s 3ms/sample - loss: 0.0400 - accuracy: 0.9861
Epoch 5/10
14942/14942 [==============================] - 41s 3ms/sample - loss: 0.0207 - accuracy: 0.9933
Epoch 6/10
14942/14942 [==============================] - 41s 3ms/sample - loss: 0.0226 - accuracy: 0.9930
Epoch 7/10
14942/14942 [==============================] - 42s 3ms/sample - loss: 0.0149 - accuracy: 0.9946
Epoch 8/10
14942/14942 [==============================] - 41s 3ms/sample - loss: 0.0095 - accuracy: 0.9974
Epoch 9/10
14942/14942 [==============================] - 41s 3ms/sample - loss: 0.0144 - accuracy: 0.9949
Epoch 10/10
14

## Pruebas del nuevo modelo con data sin ruido

In [21]:
print(new_model.evaluate(X_test, y_test))

[0.10713661153884232, 0.9716274]


## Pruebas del nuevo modelo con data con ruido

In [22]:
new_classifier = KerasClassifier(model=new_model)

In [23]:
epsilon = 10
adv_crafter = FastGradientMethod(og_classifier, eps=epsilon)
x_test_adv = adv_crafter.generate(x=X_test)

In [24]:
preds = np.argmax(new_classifier.predict(x_test_adv), axis =1)
acc = np.sum(preds == np.argmax(y_test,axis =1))/ y_test.shape[0]
print("\nTest accuracy on adversarial sample: %.2f%%" % (acc * 100))


Test accuracy on adversarial sample: 84.42%
